From [Christorpher GS Blog](https://christophergs.com/blog/ai-engineering-retrieval-augmented-generation-rag-llama-index)

Definitely needs a GPU in Colab

In [1]:
# get model

# download the Trial by Sorcery file from here https://manybooks.net/titles/trial-by-sorcery

# get the model

!pip3 install huggingface-hub
!huggingface-cli download TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmp1veqz8cg
mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf: 100% 26.4G/26.4G [03:32<00:00, 124MB/s]
./mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf


In [2]:
%pip install llama_index
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# cpu based
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import os
os.environ["OMP_NUM_THREADS"] = "10"

llm = LlamaCPP(
    model_path="./mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf",  # https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF
    context_window=32000,
    max_new_tokens=1024,
    verbose=True
)

embedding_model = HuggingFaceEmbedding(model_name="WhereIsAI/UAE-Large-V1")  # https://huggingface.co/WhereIsAI/UAE-Large-V1

llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from ./mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mixtral-8x7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:            

In [3]:
# # gpu based
# from llama_index.llms.llama_cpp import LlamaCPP
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# llm = LlamaCPP(
#     model_path="./mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf",  # https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF
#     context_window=32000,
#     max_new_tokens=1024,
#     model_kwargs={'n_gpu_layers': 1},
#     verbose=True
# )

# embedding_model = HuggingFaceEmbedding(model_name="WhereIsAI/UAE-Large-V1")  # https://huggingface.co/WhereIsAI/UAE-Large-V1

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from ./mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mixtral-8x7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:            

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [2]:
from transformers import AutoTokenizer
from llama_index.core import Settings

Settings.tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [3]:
import logging
import sys
from llama_index.core import set_global_handler

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
set_global_handler("simple")

In [4]:
%%time

from pathlib import Path
import glob
from llama_index.core import SimpleDirectoryReader, ServiceContext, StorageContext, VectorStoreIndex, download_loader
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

PDFReader = download_loader('PDFReader')
loader = PDFReader()

documents = loader.load_data(file=Path("./Trial-by-Sorcery.pdf"))

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_model,
    system_prompt='You are a bot that answers questions about the book.'
)

# create client and a new collection
chroma_client = chromadb.EphemeralClient()


chroma_collection = chroma_client.get_or_create_collection("quickstart")
chroma_client.delete_collection("quickstart")
chroma_collection = chroma_client.get_or_create_collection("quickstart")

# set up ChromaVectorStore and load in data
vector_store2 = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context2 = StorageContext.from_defaults(vector_store=vector_store2)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context2, embed_model=embedding_model, service_context=service_context
)

# Query Data
query_engine = index.as_query_engine()

DEBUG:chromadb.auth.registry:Registering provider: token_config
Registering provider: token_config
DEBUG:chromadb.auth.registry:Registering provider: user_token_config
Registering provider: user_token_config
DEBUG:chromadb.auth.registry:Registering provider: token
Registering provider: token
DEBUG:chromadb.auth.registry:Registering provider: token
Registering provider: token


<timed exec>:7: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
<timed exec>:12: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.


INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
DEBUG:chromadb.config:Starting component System
Starting component System
DEBUG:chromadb.config:Starting component Posthog
Starting component Posthog
DEBUG:chromadb.config:Starting component OpenTelemetryClient
Starting component OpenTelemetryClient
DEBUG:chromadb.config:Starting component SimpleAssignmentPolicy
Starting component SimpleAssignmentPolicy
DEBUG:chromadb.config:Starting component SqliteDB
Starting component SqliteDB
DEBUG:chromadb.config:Starting component QuotaEnforcer
Starting component QuotaEnforcer
DEBUG:chromadb.config:Starting component LocalSegmentManager
Starting component LocalSegmentManager
DEBUG:chromadb.config:Starting component SegmentAPI
Starting component SegmentAPI
DEBUG:llama_index.core.node_pa

In [5]:
%%time
response = query_engine.query("What is a Dragon Guard?")

DEBUG:llama_index.vector_stores.chroma.base:> Top 1 nodes:
> Top 1 nodes:
DEBUG:llama_index.vector_stores.chroma.base:> [Node f63e8ccc-d890-476d-923f-1b91637d1ebc] [Similarity score: 0.5236360740421369] 2
	
The	entrance	to	the	Citadel	was	much	more	heavily	guarded	than	the	city
gates.	And	these	guar...
> [Node f63e8ccc-d890-476d-923f-1b91637d1ebc] [Similarity score: 0.5236360740421369] 2
	
The	entrance	to	the	Citadel	was	much	more	heavily	guarded	than	the	city
gates.	And	these	guar...
DEBUG:llama_index.vector_stores.chroma.base:> [Node ff614cd2-0e73-45dd-a50b-451d0cc90501] [Similarity score: 0.46177424703594677] 1
	
I	marveled	at	the	vastness	of	the	Citadel.
It	was	home	to	the	Dragon	Guard,	the	greatest	warr...
> [Node ff614cd2-0e73-45dd-a50b-451d0cc90501] [Similarity score: 0.46177424703594677] 1
	
I	marveled	at	the	vastness	of	the	Citadel.
It	was	home	to	the	Dragon	Guard,	the	greatest	warr...
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node f63e8ccc-d890-476d-923f-1b91637d


llama_print_timings:        load time =  319191.34 ms
llama_print_timings:      sample time =      52.61 ms /    81 runs   (    0.65 ms per token,  1539.72 tokens per second)
llama_print_timings: prompt eval time = 1238109.03 ms /  1899 tokens (  651.98 ms per token,     1.53 tokens per second)
llama_print_timings:        eval time =  226191.64 ms /    80 runs   ( 2827.40 ms per token,     0.35 tokens per second)
llama_print_timings:       total time = 1466429.76 ms /  1979 tokens


** Prompt: **
You are a bot that answers questions about the book.

Context information is below.
---------------------
page_label: 11
file_name: Trial-by-Sorcery.pdf

2
	
The	entrance	to	the	Citadel	was	much	more	heavily	guarded	than	the	city
gates.	And	these	guards	weren’t	the	city	guard,	either.	They	were	Dragon
Guards.	Their	armor	was	decorated	to	look	like	dragon	scales,	but	it	was
versatile	and	practical	for	battle.	Behind	the	assemblage	of	guards	was	a	long
wooden	table	that	had	weapons	scattered	haphazardly	on	its	surface.
As	I	drew	nearer,	there	was	a	
whooshing
	sound	that	echoed	off	the	massive
walls	and	made	the	items	on	the	table	clatter.	The	guards	seemed
unperturbed	by	the	noise,	but	I	was	trying	to	figure	out	what	it	was	and
where	it	was	coming	from.	Suddenly,	a	massive	blue	dragon	swooped	down
from	the	sky	and	landed	in	the	courtyard.
I	held	my	breath	in	awe	as	I	stared	at	the	powerful	beast.	It	was	easily
thirty	feet	long	from	its	nose	to	its	tail.	The	dragon’s	rider	